In [1]:
%matplotlib inline

# PyClang boostraping

In [2]:
includedir = "/usr/local/include"

headers = ["clang/AST/ExternalASTSource.h", "clang/AST/DependentDiagnostic.h", "clang/AST/ParentMap.h",
           "clang/AST/DeclGroup.h", "clang/AST/DeclAccessPair.h", "clang/AST/UnresolvedSet.h",
           "clang/AST/DeclarationName.h", "clang/AST/Stmt.h", "clang/AST/Redeclarable.h", "clang/AST/Type.h",
           "clang/AST/SelectorLocationsKind.h", "clang/AST/ExprObjC.h", "clang/AST/OperationKinds.h",
           "clang/AST/DeclObjC.h", "clang/AST/CXXInheritance.h", "clang/AST/CommentLexer.h",
           "clang/AST/ASTTypeTraits.h", "clang/AST/TemplateBase.h", "clang/AST/DeclFriend.h", 
           "clang/AST/OpenMPClause.h", "clang/AST/DeclLookups.h", "clang/AST/StmtVisitor.h",
           "clang/AST/VTableBuilder.h", "clang/AST/ASTUnresolvedSet.h", "clang/AST/GlobalDecl.h",
           "clang/AST/DeclBase.h", "clang/AST/ASTContext.h", "clang/AST/CharUnits.h", "clang/AST/StmtOpenMP.h",
           "clang/AST/TypeLocVisitor.h", "clang/AST/LambdaCapture.h", "clang/AST/CanonicalType.h",
           "clang/AST/PrettyPrinter.h", "clang/AST/CommentCommandTraits.h", "clang/AST/NestedNameSpecifier.h",
           "clang/AST/DeclTemplate.h", "clang/AST/Mangle.h", "clang/AST/StmtIterator.h", "clang/AST/Attr.h",
           "clang/AST/StmtGraphTraits.h", "clang/AST/MangleNumberingContext.h", "clang/AST/StmtCXX.h",
           "clang/AST/CommentSema.h", "clang/AST/RecursiveASTVisitor.h", "clang/AST/ASTConsumer.h",
           "clang/AST/ASTFwd.h", "clang/AST/TypeLoc.h", "clang/AST/CommentDiagnostic.h",
           "clang/AST/VTTBuilder.h", "clang/AST/EvaluatedExprVisitor.h", "clang/AST/CommentParser.h",
           "clang/AST/AttrIterator.h", "clang/AST/DataRecursiveASTVisitor.h", "clang/AST/NSAPI.h",
           "clang/AST/TypeOrdering.h", "clang/AST/TypeVisitor.h", "clang/AST/Decl.h", "clang/AST/ExprCXX.h",
           "clang/AST/CommentVisitor.h", "clang/AST/ASTDiagnostic.h", "clang/AST/AST.h", "clang/AST/ASTLambda.h",
           "clang/AST/StmtObjC.h", "clang/AST/RecordLayout.h", "clang/AST/APValue.h",
           "clang/AST/DeclContextInternals.h", "clang/AST/TemplateName.h", "clang/AST/ASTImporter.h",
           "clang/AST/DeclOpenMP.h", "clang/AST/ASTMutationListener.h", "clang/AST/RawCommentList.h",
           "clang/AST/ASTVector.h", "clang/AST/Comment.h", "clang/AST/DeclCXX.h", "clang/AST/CommentBriefParser.h",
           "clang/AST/Expr.h", "clang/AST/BaseSubobject.h", "clang/AST/DeclVisitor.h", "clang/Frontend/ASTUnit.h",
           "clang/Basic/Specifiers.h"]

headers = [includedir + '/' + header for header in headers]

In [3]:
from vplants.autowig.asg import *
asg = AbstractSemanticGraph()
for header in headers:
    filenode = asg.add_file(header, language='c++')

In [4]:
flags = ['-x', 'c++', '-g', '-std=c++11', '-I'+includedir, '-I/usr/local/lib/clang/3.7.0/include',
         '-D__STDC_LIMIT_MACROS', '-D__STDC_CONSTANT_MACROS']
asg[includedir + '/'].parse(flags=flags, libclang=False)

/home/pfernique/Desktop/GitHub/pfernique/autowig/src/vplants/autowig/asg.py:2376: NotImplementedTypeWarning: 'ConstantArray' for field '::pthread_attr_t::__size'
  warnings.warn(str(warning) + ' for field \'' + spelling + '\'', warning.__class__)
/home/pfernique/Desktop/GitHub/pfernique/autowig/src/vplants/autowig/asg.py:2376: NotImplementedTypeWarning: 'ConstantArray' for field '::__size'
  warnings.warn(str(warning) + ' for field \'' + spelling + '\'', warning.__class__)
/home/pfernique/Desktop/GitHub/pfernique/autowig/src/vplants/autowig/asg.py:2376: AnonymousClassWarning: in scope '' for field '::__data'
  warnings.warn(str(warning) + ' for field \'' + spelling + '\'', warning.__class__)
/home/pfernique/Desktop/GitHub/pfernique/autowig/src/vplants/autowig/asg.py:2376: NotImplementedTypeWarning: 'ConstantArray' for field '::__size'
  warnings.warn(str(warning) + ' for field \'' + spelling + '\'', warning.__class__)
/home/pfernique/Desktop/GitHub/pfernique/autowig/src/vplants/autowig

In [5]:
asg.clean()

Previous number of nodes: 93614
Current number of nodes: 25698
Percentage of nodes cleaned: 72.55%

In [6]:
from vplants.autowig.bootstrap import *
clang_tooling, clang_cast = bootstrap(asg)
print 'Boostrapped nodes:', len(asg)

Boostrapped nodes: 455


In [7]:
decl = asg.classes('^(class |struct |)::clang::Decl$').pop()
decl.is_copyable = False
for inheritor in decl.inheritors(True):
    inheritor.is_copyable = False

In [8]:
from vplants.autowig.boost_python import *
    
module = asg.boost_python_module('../src/boost_python/autowig.cpp', '(.*)')
filenode = asg.add_file('../src/boost_python/export_functions_clang_tooling.cpp')
filenode.is_protected = False
filenode.content = clang_tooling
module.add_export(filenode)
filenode = asg.add_file('../src/boost_python/export_functions_clang_cast.cpp')
filenode.is_protected = False
filenode.content = clang_cast
module.add_export(filenode)
module.write()

In [9]:
from vplants.autowig.scons import *
asg.add_sconstruct('../')
asg.scons('-j', '2')